In [10]:
import numpy as np
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# (데이터 출처: 캐글)
- 링크: [Rock Paper Scissors Dataset](https://www.kaggle.com/datasets/sanikamal/rock-paper-scissors-dataset)
- train 데이터 2,520개, test 데이터 372개
- 원본 이미지 크기는 300x300이나 연산 수행 소요시간 및 메모리 관리를 위해 150x150로 축소
- 원본 이미지는 RGBA의 4채널이나, 연산 수행 소요시간 및 메모리 관리를 위해 RGB 3채널로 축소

In [12]:
# 데이터 경로 설정
train_dir = "/content/drive/My Drive/rock_scissors_paper/train"
test_dir = "/content/drive/My Drive/rock_scissors_paper/test"

# 1. 학습 데이터에 대한 데이터 증강

- 이미지를 회전, 이동, 확대/축소 등의 방법으로 변형하여 데이터를 증강
- validation_split=0.2는 학습 데이터의 20%를 검증 데이터로 사용한다는 의미

In [13]:
# 데이터 증강 및 로딩
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 검증 데이터 분리
)

# 테스트 데이터에 대한 전처리 설정, 픽셀값을 0~1 사이로 조정(정규화 과정)
test_datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터를 로드하는 제너레이터
# 이미지 크기를 150x150으로 조정하고, 배치 크기는 32로 설정
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

# 검증 데이터를 로드하는 제너레이터
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# 테스트 데이터를 로드하는 제너레이터
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


# 2. 모델 정의

In [14]:
n_channel_1 = 16
n_channel_2 = 32
n_channel_3 = 64
n_dense = 32
num_classes = 3

model = Sequential([
    Conv2D(n_channel_1, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(n_channel_2, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(n_dense, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# 모델 컴파일, 옵티마이저, 손실 함수, 평가 지표를 지정
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 3. 모델 학습

In [16]:
n_train_epoch = 20

# 조기 종료(Early Stopping) 콜백을 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    train_generator,
    epochs=n_train_epoch,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.3250 - loss: 1.3649 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.3513 - loss: 1.0986 - val_accuracy: 0.3333 - val_loss: 1.0985
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.3318 - loss: 1.0991 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.3295 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.3242 - loss: 1.0986 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.3203 - loss: 1.0986 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.3311 - loss: 1.0987 - val_accuracy: 0.3333 - val_loss: 1.0986
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.3241 - loss: 1.0986 - val_accuracy: 0.3333 - val_los

# 4. 성능 평가

In [17]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

12/12 - 61s - 5s/step - accuracy: 0.3333 - loss: 1.0985
Test Loss: 1.0985
Test Accuracy: 0.3333
